In [28]:
## load dependencies
import pandas as pd
import geopandas as gpd
import requests
from PIL import Image, ImageDraw
from io import BytesIO

In [2]:
## load data
df = pd.read_csv('../data/campustreedata.csv')

#remove trees with 0 lat or lon (indicating no location is recorded)
df = df[(df.LAT)!=0]
df = df[(df.LONG)!=0]

#load map zone data (this was manually created by me)
zone_gdf = gpd.read_file('../shapefiles/map.geojson')
zone_gdf = zone_gdf.to_crs("EPSG:4326")

In [45]:
## Download images of all the trees from ESRI
def get_tree_images(df=df, buffer=.0001, limit=None, x_pix_len=64, y_pix_len=64):
    count = 0
    for i, row in df.iterrows():
        FID = str(row['FID'])
        x, y = row['LONG'], row['LAT']
        minx, maxx = x-buffer, x+buffer
        miny, maxy = y-buffer,y+buffer
        esri_url = f"https://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/export?bbox={minx},{miny},{maxx},{maxy}&bboxSR=4326&imageSR=4326&size={x_pix_len},{y_pix_len}&f=image"
        response = requests.get(esri_url)
        filename = '../Tree_Images/FID_' + FID + '.png'
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content))
            img.save(filename)
        else:
            print("Failed to retrieve picure of tree FID " + FID + ".")
        count += 1
        if count % 100 == 0:
            print("Loaded image", count, "of", len(df), "...")
        if count == limit:
            break
    return(img, "Done!")

In [51]:
get_tree_images(df=df[17475:], limit=None, x_pix_len=80, y_pix_len=80)

(<PIL.PngImagePlugin.PngImageFile image mode=P size=80x80>, 'Done!')